In [63]:
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

# Récupération de la data

In [64]:
driver = webdriver.Chrome()
driver.get("https://www.boursier.com/actions/paris")
time.sleep(1)
div_refresh_init = driver.find_element(By.CLASS_NAME, "refresh-init")
div_tab = div_refresh_init.find_element(By.CLASS_NAME, "scrolly-table")

tableau_data = div_tab.find_element(By.TAG_NAME, 'table')
tableau_data_body = div_tab.find_element(By.TAG_NAME, 'tbody')

liste_ligne_tab = tableau_data_body.find_elements(By.TAG_NAME, 'tr')
data = []
for col in liste_ligne_tab:
    raw = col.find_elements(By.TAG_NAME, 'td')
    liste_data_raw = []
    for i in raw:
        liste_data_raw.append(i.text)
    
    data.append(liste_data_raw)

driver.quit()

# Transformation de la data liste --> csv

In [65]:
liste_colonne = ['Societe', 'valeur_actuel', 'variation', 'ouverture', 'valeur_haute', 'valeur_basse', 'volume']

df = pd.DataFrame(data, columns=liste_colonne)

df.head()

,Societe,valeur_actuel,variation,ouverture,valeur_haute,valeur_basse,volume
0,AB SCIENCE,"0,99€","+1,97%","0,97€","0,99€","0,95€",25 326
1,ABC ARBITRAGE,"4,59€","-0,76%","4,62€","4,62€","4,54€",13 000
2,ABIONYX PHARMA,"1,23€","-0,65%","1,24€","1,24€","1,20€",63 611
3,ABIVAX,"9,54€","-1,45%","9,60€","9,60€","9,38€",5 713
4,ACCOR,"42,28€","+0,19%","42,13€","42,72€","42,13€",263 379


In [66]:
def transform_data(dataframe, colonne, type):
    # Remplacer tous les caractères non numériques (excepté le point) par rien
    df = dataframe[colonne].str.replace(r'[€,\+\-%]', '', regex=True).str.strip().astype(type)
    return df


In [67]:
df['valeur_actuel'] = transform_data(df, 'valeur_actuel', float)
df['variation'] = transform_data(df, 'variation', float)
df['ouverture'] = transform_data(df, 'ouverture', float)
df['valeur_haute'] = transform_data(df, 'valeur_haute', float)
df['valeur_basse'] = transform_data(df, 'valeur_basse', float)
df.head()

,Societe,valeur_actuel,variation,ouverture,valeur_haute,valeur_basse,volume
0,AB SCIENCE,99.0,197.0,97.0,99.0,95.0,25 326
1,ABC ARBITRAGE,459.0,76.0,462.0,462.0,454.0,13 000
2,ABIONYX PHARMA,123.0,65.0,124.0,124.0,120.0,63 611
3,ABIVAX,954.0,145.0,960.0,960.0,938.0,5 713
4,ACCOR,4228.0,19.0,4213.0,4272.0,4213.0,263 379
